In [1]:
## Import relevant modules

from numpy import*
from matplotlib.pyplot import*
from iris import*
from iris.analysis import*
import iris.quickplot as quickplot
from iris.coord_categorisation import*
from scipy.stats import*
from scipy.stats.mstats import*
import netCDF4 as nc
import numpy.ma as ma

In [2]:
miroc6_historical = iris.load_cube('/home/cj362/ENSO_data/tos/miroc6_historical/tos_Omon_MIROC6_historical_r1i1p1f1_gn_195001-201412_regridded.nc')
#print(miroc6_historical)

# Define Nino3.4 region
north=5
east=-120
south=-5
west=-170

# Subset files to Nino3.4 region
miroc6_historical_lon = miroc6_historical.intersection(longitude =(west,east))
miroc6_historical_nino34=miroc6_historical_lon.intersection(latitude=(south,north))

## Monthly mean SST across Nino3.4 region 2015-2100 (1032 months = 1032 output values)
miroc6_historical_nino34.coord('longitude').guess_bounds() # Define longitude bounds
miroc6_historical_nino34.coord('latitude').guess_bounds() # Define latitude bounds
miroc6_historical_nino34_mon_mean_grid_area = iris.analysis.cartography.area_weights(miroc6_historical_nino34) # Create grid area
miroc6_historical_nino34_mon_mean = miroc6_historical_nino34.collapsed(['longitude', 'latitude'],\
                              iris.analysis.MEAN, weights=miroc6_historical_nino34_mon_mean_grid_area) # Collape values

# print(miroc6_historical_nino34_mon_mean) # Print collapsed file

miroc6_historical_nino34_mon_mean_data = miroc6_historical_nino34_mon_mean.data # Extract collapsed file data
print(miroc6_historical_nino34_mon_mean_data)
print

## Save monthly mean SST array as .txt file source: https://www.kite.com/python/examples/5522/numpy-save-an-array-to-a-text-file
# Format: np.savetxt("outputfilename",np.specified array you want to save)
np.savetxt("miroc6_historical_nino34_mon_mean_data.txt",np.array(miroc6_historical_nino34_mon_mean_data))


[25.32474632 25.83310178 26.34852826 27.1255497  27.37143072 27.17676263
 26.52116229 25.62509165 25.3603995  25.05208333 25.18982756 25.38753286
 25.48424766 25.70427341 26.3992915  27.1944176  27.25774265 27.29479589
 26.82446643 26.26107446 25.56397257 25.41505469 25.1606249  25.26574653
 25.46041243 25.86290067 26.54714622 27.57624874 27.27601188 26.95565086
 26.46940784 25.10334477 24.62435845 24.53433312 24.00797303 24.1733722
 24.72792884 25.0602154  25.34307555 25.82217694 26.05153954 25.91473698
 25.14488152 24.22615352 23.82840021 23.82779471 23.98282411 24.07819172
 24.60148363 24.99411322 25.78430948 26.18685593 26.5774529  26.66139389
 26.44600446 26.1139746  25.89489605 25.84213937 25.50886897 25.40495981
 25.48129229 25.82247646 26.50822228 27.12881471 27.65796463 27.59646528
 27.49738157 27.17761391 26.95484625 26.95311697 26.7043212  26.63522163
 26.54149415 26.65425062 27.13891936 27.52175157 28.03831101 28.04671004
 27.70788134 27.16942102 26.5416391  26.16753708 26.

/home/cj362/anaconda3/envs/python27/lib/python2.7/site-packages/iris/fileformats/cf.py:803: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'tos'
  warnings.warn(message % (variable_name, nc_var_name))
/home/cj362/anaconda3/envs/python27/lib/python2.7/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


In [3]:
# Load in climate netCDF files

miroc6_ssp2 = iris.load_cube('/home/cj362/ENSO_data/tos/miroc6_ssp2_45/tos_Omon_MIROC6_ssp245_r1i1p1f1_gn_201501-210012_reggridded.nc')

# print(miroc6_ssp2)

# Define Nino3.4 region
north=5
east=-120
south=-5
west=-170

# Subset files to Nino3.4 region
miroc6_ssp2_lon=miroc6_ssp2.intersection(longitude =(west,east))
miroc6_ssp2_nino34=miroc6_ssp2_lon.intersection(latitude=(south,north))


##  Mean SST for each grid cell between 2015-2100 (714 grid squares = 714 output values)
# miroc6_ssp2_nino34_gridsquare_mean = miroc6_ssp2_nino34.collapsed('time', iris.analysis.MEAN) 
# print(miroc6_ssp2_nino34_gridsquare_mean)
# miroc6_ssp2_nino34_mgridsquare_mean_data = miroc6_ssp2_nino34_gridsquare_mean.data
# print(miroc6_ssp2_nino34_gridsquare_mean_data)

## Monthly mean SST across Nino3.4 region 2015-2100 (1032 months = 1032 output values)
miroc6_ssp2_nino34.coord('longitude').guess_bounds() # Define longitude bounds
miroc6_ssp2_nino34.coord('latitude').guess_bounds() # Define latitude bounds
miroc6_ssp2_nino34_mon_mean_grid_area = iris.analysis.cartography.area_weights(miroc6_ssp2_nino34) # Create grid area
miroc6_ssp2_nino34_mon_mean = miroc6_ssp2_nino34.collapsed(['longitude', 'latitude'],\
                              iris.analysis.MEAN, weights=miroc6_ssp2_nino34_mon_mean_grid_area) # Collape values

# print(miroc6_ssp2_nino34_mon_mean) # Print collapsed file

miroc6_ssp2_nino34_mon_mean_data = miroc6_ssp2_nino34_mon_mean.data # Extract collapsed file data
print(miroc6_ssp2_nino34_mon_mean_data)
print
# print(type(miroc6_ssp2_nino34_mon_mean_data))



## Quick, non-truncated view of all entries within monthly mean SST array
# for i in range (0, len(miroc6_ssp2_nino34_mon_mean_data)):
    
    # print(miroc6_ssp2_nino34_mon_mean_data[i])
    

# View number of dimensions, shape, size of numpy source: https://note.nkmk.me/en/python-numpy-ndarray-ndim-shape-size/
# array_name.ndim - dimensions
# array_name.shape - shape
# array_name.size - size (multiply sides of the shape)


## Save monthly mean SST array as .txt file source: https://www.kite.com/python/examples/5522/numpy-save-an-array-to-a-text-file
# Format: np.savetxt("outputfilename",np.specified array you want to save)
np.savetxt("miroc6_ssp2_nino34_mon_mean_data.txt",np.array(miroc6_ssp2_nino34_mon_mean_data))





## Identifying ENSO using ONI

# 

[23.76083856 24.34810953 24.45163283 ... 29.19931583 29.1406727
 28.8058407 ]

